# Análisis Exploratorio de Sentimientos

Este notebook contiene análisis exploratorio de los datos de sentimientos procesados por el pipeline.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from clickhouse_driver import Client
from datetime import datetime, timedelta

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline

In [ ]:
# Conectar a ClickHouse
client = Client(
    host='localhost',
    port=9000,
    user='admin',
    password='password',
    database='sentiment_db'
)

## 1. Análisis General de Volumen

In [ ]:
# Obtener datos de las últimas 24 horas
query = """
SELECT 
    toStartOfHour(created_at) as hour,
    count() as total_tweets,
    countIf(predicted_sentiment = 'positive') as positive,
    countIf(predicted_sentiment = 'neutral') as neutral,
    countIf(predicted_sentiment = 'negative') as negative
FROM sentiment_analysis 
WHERE created_at >= now() - INTERVAL 24 HOUR
GROUP BY hour
ORDER BY hour
"""

df_hourly = pd.DataFrame(client.execute(query), 
                        columns=['hour', 'total_tweets', 'positive', 'neutral', 'negative'])
df_hourly['hour'] = pd.to_datetime(df_hourly['hour'])
df_hourly.head()

In [ ]:
# Visualizar tendencias por hora
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Volumen total
ax1.plot(df_hourly['hour'], df_hourly['total_tweets'], marker='o', linewidth=2)
ax1.set_title('Volumen de Tweets por Hora', fontsize=16)
ax1.set_ylabel('Número de Tweets')
ax1.grid(True, alpha=0.3)

# Distribución de sentimientos
ax2.plot(df_hourly['hour'], df_hourly['positive'], marker='o', label='Positivo', color='green')
ax2.plot(df_hourly['hour'], df_hourly['neutral'], marker='s', label='Neutral', color='gray')
ax2.plot(df_hourly['hour'], df_hourly['negative'], marker='^', label='Negativo', color='red')
ax2.set_title('Distribución de Sentimientos por Hora', fontsize=16)
ax2.set_ylabel('Número de Tweets')
ax2.set_xlabel('Hora')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Análisis de Confianza del Modelo

In [ ]:
# Análisis de confianza por sentimiento
query_confidence = """
SELECT 
    predicted_sentiment,
    avg(confidence) as avg_confidence,
    quantile(0.5)(confidence) as median_confidence,
    min(confidence) as min_confidence,
    max(confidence) as max_confidence,
    count() as count
FROM sentiment_analysis 
WHERE created_at >= now() - INTERVAL 24 HOUR
GROUP BY predicted_sentiment
"""

df_confidence = pd.DataFrame(client.execute(query_confidence),
                           columns=['sentiment', 'avg_confidence', 'median_confidence', 
                                  'min_confidence', 'max_confidence', 'count'])
print("Estadísticas de Confianza por Sentimiento:")
print(df_confidence)

## 3. Análisis de Engagement

In [ ]:
# Relación entre sentimiento y engagement
query_engagement = """
SELECT 
    predicted_sentiment,
    avg(like_count) as avg_likes,
    avg(retweet_count) as avg_retweets,
    avg(like_count + retweet_count) as avg_total_engagement
FROM sentiment_analysis 
WHERE created_at >= now() - INTERVAL 24 HOUR
    AND (like_count > 0 OR retweet_count > 0)
GROUP BY predicted_sentiment
"""

df_engagement = pd.DataFrame(client.execute(query_engagement),
                           columns=['sentiment', 'avg_likes', 'avg_retweets', 'avg_total_engagement'])

# Visualizar engagement por sentimiento
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
x = np.arange(len(df_engagement))
width = 0.35

ax.bar(x - width/2, df_engagement['avg_likes'], width, label='Likes', alpha=0.8)
ax.bar(x + width/2, df_engagement['avg_retweets'], width, label='Retweets', alpha=0.8)

ax.set_xlabel('Sentimiento')
ax.set_ylabel('Engagement Promedio')
ax.set_title('Engagement Promedio por Sentimiento')
ax.set_xticks(x)
ax.set_xticklabels(df_engagement['sentiment'])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Palabras Clave Trending

In [ ]:
# Top palabras por sentimiento
query_words = """
WITH word_sentiment AS (
    SELECT 
        arrayJoin(extractAll(lower(text), '[a-zA-Z]{4,}')) as word,
        predicted_sentiment
    FROM sentiment_analysis 
    WHERE created_at >= now() - INTERVAL 24 HOUR
        AND length(word) >= 4
        AND word NOT IN ('this', 'that', 'with', 'have', 'will', 'from', 'they', 'been', 'were', 'said', 'each', 'which', 'their', 'time', 'would', 'there', 'could', 'other', 'more', 'very', 'what', 'know', 'just', 'first', 'into', 'over', 'think', 'also', 'your', 'work', 'life', 'only', 'can', 'still', 'should', 'after', 'being', 'now', 'made', 'before', 'here', 'through', 'when', 'where', 'much', 'some', 'these', 'many', 'then', 'them', 'well', 'were')
)
SELECT 
    word,
    predicted_sentiment,
    count() as frequency
FROM word_sentiment
GROUP BY word, predicted_sentiment
HAVING frequency >= 3
ORDER BY frequency DESC
LIMIT 50
"""

df_words = pd.DataFrame(client.execute(query_words),
                       columns=['word', 'sentiment', 'frequency'])

# Top 10 palabras por cada sentimiento
for sentiment in ['positive', 'negative', 'neutral']:
    top_words = df_words[df_words['sentiment'] == sentiment].head(10)
    print(f"\nTop 10 palabras - {sentiment.upper()}:")
    for _, row in top_words.iterrows():
        print(f"  {row['word']}: {row['frequency']} menciones")

## 5. Análisis Temporal Detallado

In [ ]:
# Análisis por día de la semana
query_weekday = """
SELECT 
    toDayOfWeek(created_at) as day_of_week,
    count() as total_tweets,
    avg(confidence) as avg_confidence,
    countIf(predicted_sentiment = 'positive') / count() as positive_rate
FROM sentiment_analysis 
WHERE created_at >= now() - INTERVAL 7 DAY
GROUP BY day_of_week
ORDER BY day_of_week
"""

df_weekday = pd.DataFrame(client.execute(query_weekday),
                         columns=['day_of_week', 'total_tweets', 'avg_confidence', 'positive_rate'])

# Mapear días de la semana
day_names = {1: 'Lunes', 2: 'Martes', 3: 'Miércoles', 4: 'Jueves', 
            5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
df_weekday['day_name'] = df_weekday['day_of_week'].map(day_names)

print("Análisis por día de la semana:")
print(df_weekday[['day_name', 'total_tweets', 'avg_confidence', 'positive_rate']])

## 6. Resumen y Conclusiones

In [ ]:
# Generar resumen ejecutivo
query_summary = """
SELECT 
    count() as total_tweets,
    countIf(predicted_sentiment = 'positive') / count() * 100 as positive_pct,
    countIf(predicted_sentiment = 'negative') / count() * 100 as negative_pct,
    avg(confidence) as avg_confidence,
    sum(like_count + retweet_count) as total_engagement,
    uniq(author_id) as unique_authors
FROM sentiment_analysis 
WHERE created_at >= now() - INTERVAL 24 HOUR
"""

summary = client.execute(query_summary)[0]

print("\n" + "="*60)
print("📊 RESUMEN EJECUTIVO - ÚLTIMAS 24 HORAS")
print("="*60)
print(f"📈 Total de tweets procesados: {summary[0]:,}")
print(f"😊 Sentimiento positivo: {summary[1]:.1f}%")
print(f"😞 Sentimiento negativo: {summary[2]:.1f}%")
print(f"🎯 Confianza promedio del modelo: {summary[3]:.3f}")
print(f"💫 Engagement total: {summary[4]:,}")
print(f"👥 Autores únicos: {summary[5]:,}")
print("="*60)